<a href="https://colab.research.google.com/github/purnimapatel/Comparing-of-Deep_learning-Neural-Network/blob/main/LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the libraries
import numpy as np
import pandas as pd
#from numpy import mean
#from numpy import std
#from numpy import dstack
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout

In [2]:
#opening the zip-file
from zipfile import ZipFile
file_name1="UCI HAR Dataset.zip"
with ZipFile(file_name1,'r') as zip:
  zip.extractall()
  print("DONE_1")


DONE_1


In [3]:
#those are separate normalised input features for the neural network
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]
# Output classes to learn how to classify
LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

In [4]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load_signals
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        #/content/UCI HAR Dataset/train/Inertial Signals
        #/content/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [5]:
def load_y(subset):
    filename = f'/content/UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [6]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, y_train, X_test,  y_test

In [7]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [8]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [9]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [10]:
# Loading the train and test data
X_train, Y_train, X_test,  Y_test = load_data()

In [11]:
print(X_train.shape)

(7352, 128, 9)


In [12]:
print(X_test.shape)

(2947, 128, 9)


In [13]:
print(Y_train.shape)

(7352, 6)


In [14]:
print(Y_test.shape)

(2947, 6)


In [15]:
training_data_count = len(X_train)
test_data_count = len(X_test) 
print(training_data_count)
print(test_data_count)

7352
2947


# LSTM_RNN


In [16]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [17]:
# initialising the parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [19]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)
#n_classes be 6
print(timesteps)
print(input_dim)
print(len(X_train))
print(n_classes)

128
9
7352
6


**Base_model of LSTM-RNN**


In [20]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [22]:
# Training the model
model.fit(X_train,Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs,shuffle=True)

Epoch 1/30
460/460 [==============================] - 19s 41ms/step - loss: 1.4152 - accuracy: 0.3490 - val_loss: 1.3627 - val_accuracy: 0.3984
Epoch 2/30
460/460 [==============================] - 19s 40ms/step - loss: 1.0927 - accuracy: 0.5469 - val_loss: 0.9226 - val_accuracy: 0.6227
Epoch 3/30
460/460 [==============================] - 18s 40ms/step - loss: 0.8244 - accuracy: 0.6496 - val_loss: 0.7819 - val_accuracy: 0.6352
Epoch 4/30
460/460 [==============================] - 19s 41ms/step - loss: 0.7127 - accuracy: 0.6742 - val_loss: 0.7771 - val_accuracy: 0.6535
Epoch 5/30
460/460 [==============================] - 19s 41ms/step - loss: 0.6585 - accuracy: 0.6950 - val_loss: 0.7408 - val_accuracy: 0.6851
Epoch 6/30
460/460 [==============================] - 19s 42ms/step - loss: 0.5801 - accuracy: 0.7552 - val_loss: 0.6393 - val_accuracy: 0.7448
Epoch 7/30
460/460 [==============================] - 19s 40ms/step - loss: 0.4957 - accuracy: 0.8108 - val_loss: 0.4658 - val_accuracy:

In [23]:
Y_pred = model.predict(X_test)

In [24]:
pred = np.argmax(Y_pred,axis = 1) 
Y_actual = np.argmax(Y_test,axis = 1)

In [26]:
from sklearn.metrics import classification_report, confusion_matrix

In [27]:
confusion_matrix(Y_actual, pred)

array([[467,   3,  26,   0,   0,   0],
       [ 13, 452,   1,   5,   0,   0],
       [  3,  16, 401,   0,   0,   0],
       [  0,  27,   0, 413,  51,   0],
       [  4,  71,   0, 101, 356,   0],
       [  0,  27,   0,   0,   0, 510]])

In [28]:
print(classification_report(Y_actual, pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95       496
           1       0.76      0.96      0.85       471
           2       0.94      0.95      0.95       420
           3       0.80      0.84      0.82       491
           4       0.87      0.67      0.76       532
           5       1.00      0.95      0.97       537

    accuracy                           0.88      2947
   macro avg       0.89      0.89      0.88      2947
weighted avg       0.89      0.88      0.88      2947



**Multi_Layer LSTM**

In [29]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32,return_sequences=True,input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))

model.add(LSTM(28,input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.6))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 32)           5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 28)                6832      
_________________________________________________________________
dropout_2 (Dropout)          (None, 28)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 174       
Total params: 12,382
Trainable params: 12,382
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [31]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 40s 87ms/step - loss: 1.2913 - accuracy: 0.4816 - val_loss: 1.0863 - val_accuracy: 0.5691
Epoch 2/30
460/460 [==============================] - 39s 86ms/step - loss: 0.9205 - accuracy: 0.6697 - val_loss: 0.7621 - val_accuracy: 0.7061
Epoch 3/30
460/460 [==============================] - 40s 87ms/step - loss: 0.7293 - accuracy: 0.7363 - val_loss: 0.6511 - val_accuracy: 0.7272
Epoch 4/30
460/460 [==============================] - 39s 85ms/step - loss: 0.6017 - accuracy: 0.7697 - val_loss: 1.1219 - val_accuracy: 0.5928
Epoch 5/30
460/460 [==============================] - 39s 85ms/step - loss: 0.5165 - accuracy: 0.7922 - val_loss: 0.6284 - val_accuracy: 0.7594
Epoch 6/30
460/460 [==============================] - 39s 85ms/step - loss: 0.4768 - accuracy: 0.8181 - val_loss: 0.9831 - val_accuracy: 0.7323
Epoch 7/30
460/460 [==============================] - 42s 92ms/step - loss: 0.4563 - accuracy: 0.8464 - val_loss: 0.7939 - val_accuracy:

In [32]:
Y_pred = model.predict(X_test)

In [33]:
pred = np.argmax(Y_pred,axis = 1) 
Y_actual = np.argmax(Y_test,axis = 1)

In [34]:
confusion_matrix(Y_actual, pred)

array([[449,   3,  44,   0,   0,   0],
       [  5, 445,  21,   0,   0,   0],
       [  5,   5, 410,   0,   0,   0],
       [  2,   3,   0, 373, 113,   0],
       [  4,   0,   0,  59, 469,   0],
       [  0,  25,   0,   0,   2, 510]])

In [35]:
print(classification_report(Y_actual, pred))

              precision    recall  f1-score   support

           0       0.97      0.91      0.93       496
           1       0.93      0.94      0.93       471
           2       0.86      0.98      0.92       420
           3       0.86      0.76      0.81       491
           4       0.80      0.88      0.84       532
           5       1.00      0.95      0.97       537

    accuracy                           0.90      2947
   macro avg       0.90      0.90      0.90      2947
weighted avg       0.90      0.90      0.90      2947

